In [1]:
import pandas as pd
from config import *
from db_config import *
session,engine = connect_db()

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

from datetime import datetime
import json
import glob
import shutil
import os



                         ...Connecting to the database...                        


In [2]:
# import pandas as pd

# builds = ['build012','build013','build014']
# part_ids = []
# for build in builds:
#     data = pd.read_csv('{}/builds/{}/{}_seq_plate.csv'.format(BASE_PATH,build,build))
#     part_ids += data.Part.tolist()
# print(len(part_ids))

# query = "SELECT parts.part_id,fragments.fragment_name,parts.part_type,parts.seq,parts.part_name,parts.cloning_enzyme,fragments.seq FROM parts\
#             INNER JOIN part_frag ON parts.id = part_frag.part_id\
#             INNER JOIN fragments ON part_frag.fragment_id = fragments.id\
#             WHERE parts.part_id IN ({})".format(ot.list_to_string(part_ids))

# df = pd.read_sql_query(query, con=engine)

# df.to_csv('./parts_frags.csv')

# data = pd.read_csv('./parts_frags.csv')

# parts_rem = data.part_id.unique().tolist()

# for part,df in data.groupby('part_id'):
#     print(part,df.part_id.values[0])
#     for i,row in df.iterrows():
#         print(row.fragment_name)
        
# for part in part_ids:
query = "SELECT * FROM builds"

# query = "SELECT parts.part_id,fragments.fragment_name FROM parts\
#             INNER JOIN part_frag ON parts.id = part_frag.part_id\
#             INNER JOIN fragments ON part_frag.fragment_id = fragments.id"

data = pd.read_sql_query(query,con=engine)
# reused = data[data.part_id.isin(parts_rem)]
# print(len(reused.part_id.tolist()))
# reused.status.value_counts()
# to_rem = reused[reused.status == 'ordered']
# print(to_rem.id.tolist()[-1],to_rem.id.tolist()[0])
data

# current_ids = pd.read_sql_query("SELECT plates.plate_name,plates.plate_id FROM plates WHERE plates.plate_type = 'syn_plate'", con=engine)
# current_ids

,id,build_name,status,date,master_mix,seq_orders_id
0,1,build001,building,None,None,None
1,2,build002,building,None,None,None
2,3,build003,abandoned,None,None,None
3,4,build002,abandoned,None,None,None
4,5,build002,planning,None,None,None


In [ ]:
query_frag = "SELECT parts.part_id,fragments.fragment_name FROM parts\
            INNER JOIN part_frag ON parts.id = part_frag.part_id\
            INNER JOIN fragments ON part_frag.fragment_id = fragments.id"

df_frag = pd.read_sql_query(query_frag, con=engine)
frags = df_frag.groupby('part_id')['fragment_name'].agg(len)
frags.name = 'Count'
frags = pd.DataFrame(frags).reset_index()
frags_dict = dict(zip(frags.part_id.tolist(),frags.Count.tolist()))

frags_dict
frag_names = df_frag.fragment_name.tolist()
frag_names
df_frag

In [ ]:
needed = []
frags_plate = []
for file in glob.glob('{}/src/data/plate_maps/*.csv'.format(BASE_PATH)):
#     print(file)
    plate_df = pd.read_csv(file)
    frags_plate += plate_df.Name.tolist()
    if any(i in frag_names for i in plate_df.Name.tolist()):
        print('found')
        needed.append(file)
# needed = [frag for frag in frags_plate if frag in frag_names]
needed
# frags_plate

In [ ]:
df = pd.read_sql_query("SELECT * from plates", con=engine)
df